# Stable diffusion 2.1 Test # 
Written by: Jasper Bosschart
## Context ##

Testfile to test out the capabilities of huggingface models, including stabilityAI's stable diffusion.
## Sources ##
sources used for this project: 
-  https://huggingface.co/docs/diffusers/v0.16.0/en/optimization/fp16,
-  https://huggingface.co/blog/stable_diffusion#how-does-stable-diffusion-work,
-  https://huggingface.co/docs/diffusers/using-diffusers/write_own_pipeline

In [ ]:
!pip install xformers
!pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install diffusers 
!pip install accelerate 
!pip install transformers
import torch
import accelerate 
import transformers
from diffusers import StableDiffusionPipeline, UniPCMultistepScheduler

In [ ]:
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
#orch.backends.cuda.matmul.allow_tf32 = False
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
)
pipe.enable_sequential_cpu_offload()
#generator = torch.Generator("cuda").manual_seed(1024)

In [ ]:
prompt = "robot portrait, intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, 8k"
neg_prompt = "human features"


In [ ]:
prompt = "robot portrait, realistic"
neg_prompt = "painting"


In [ ]:
num_images = 4
Multi_prompt = [prompt] * num_images
Multi_prompt_N = [neg_prompt] * num_images

images = pipe(prompt=Multi_prompt, negative_prompt=Multi_prompt_N).images

grid = image_grid(images, rows=1, cols=4)
display(grid)

In [ ]:
grid.save(f"robot1.png")

In [ ]:
images = pipe(prompt=prompt, negative_prompt=neg_prompt, num_images_per_prompt=2).images

grid = image_grid(images, rows=1, cols=4)
display(grid)

In [ ]:
prompt = "portrait photograph of a robot"
image = pipe(prompt).images[0]
#image.show("image.png")
#display(image)

In [ ]:
display(grid)

In [ ]:
display(images[2])

In [ ]:
image.save(f"can1.png")